In [ ]:
# download nltk stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Install a particular version of `google-cloud-storage` because (oddly enough) 
# the  version on Colab and GCP is old. A dependency error below is okay.
!pip install -q google-cloud-storage==1.43.0

In [ ]:
# authenticate below for Google Storage access as needed
from google.colab import auth
auth.authenticate_user()

In [ ]:
# install ngrok to emulate public IP / address
!wget -N https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip

for details.

--2023-01-13 14:20:08--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.161.241.46, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  12.6MB/s    in 1.0s    

2023-01-13 14:20:10 (12.6 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
# TODO: sign up for an ngrok account
# then put your ngrok token below, uncomment, and execute
!./ngrok authtoken 22KeqcMQYpd2iRnsaZZOzlyaNw0_5NAk69pXRUMvfrf1oCNpU

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# install a ngrok python package and a version of flask that works with it in 
# colab
!pip -q install flask-ngrok
!pip -q install flask==0.12.2
# !pip -q install flask_restful

     |████████████████████████████████| 83 kB 916 kB/s 


# Run the app

In [ ]:
# you need to upload your implementation of search_app.py
import search_frontend as se

In [ ]:
# uncomment the code below and execute to reload the module when you make 
# changes to search_frontend.py (after you upload again).
# import importlib
# importlib.reload(se)

In [ ]:
from flask_ngrok import run_with_ngrok
run_with_ngrok(se.app) 
se.app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c271-34-125-136-123.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [04/Jan/2022 08:48:13] "GET /search?query=hello HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2022 08:48:13] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/Jan/2022 08:49:10] "GET /search_body?query=hello HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2022 08:49:10] "GET /favicon.ico HTTP/1.1" 404 -


# Testing your app

Once your app is running you can query it. You can simply do that by navigating to the URL that ngrok gave you above or through code in a different python session. For example, once the frontend app is running, you can navigate to:
http://YOUR_SERVER_DOMAIN/search?query=hello+world where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io, which is printed above in Colab or that is your external IP on GCP.

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [1]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
TUPLE_SIZE = 6       # We're going to pack the doc_id and tf values in this many bytes.
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer

nltk.download('stopwords')
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords) #this is containing all the stop words
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

def tokenize(text):
    """
    This function aims in tokenize a text into a list of tokens. Moreover, it filter stopwords.

    Parameters:
    -----------
    text: string , represting the text to tokenize.

    Returns:
    -----------
    list of tokens (e.g., list of tokens).
    """
    list_of_tokens = [token.group() for token in RE_WORD.finditer(text.lower()) if
                      token.group() not in all_stopwords]
    return list_of_tokens

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
map40 = {}  #all the
#dict:  k = query: v= [[recalls],[precision],avg_precision, time, query tokens quntity]

In [43]:
#recall = good/len(expected)
#precision = good/place
def map40_calculate(query, expected, predicted, duration,q_q):#map 40 for query
  map40[query] = [[],[],0,0,0]
  predicted40 = predicted[:40]
  length_expected = len(expected)
  good = 0
  place = 0
  sum_precision = 0
  for p in predicted40:
    place += 1
    # print(place)
    if p in list(expected):
      good+=1
      recall = good/length_expected
      precision = good/place
      map40[query][0].append(recall)
      map40[query][1].append(precision)
      sum_precision += precision
  #add the avg for the precision of the query
  avg_precision = sum_precision / (len(map40[query][1])+0.0000001)
  map40[query][2] = avg_precision
  map40[query][3] = duration
  map40[query][4] = q_q



# length = list(map40.items())
# def avg_map40():
#   sum_a = 0
#   for k,v in map40.items():
#     sum_a += v[2]
#     print(sum_a)
#   avg = sum_a / len(length)
#   return avg





In [5]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [6]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [44]:
import requests
from time import time
url = 'http://34.134.211.3:8080'
# place the domain you got from ngrok or GCP IP below. 
# url = 'http://XXXX-XX-XX-XX-XX.ngrok.io'

qs_res = []
arr = list(queries.items())
for i in range(len(arr)):
  q, true_wids = arr[i][0],arr[i][1]
# for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search_anchor', {'query': q}, timeout=35)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
      #our method map 40
      q_q = len(tokenize(q))
      map40_calculate(q,true_wids, pred_wids, duration,q_q)
      #################
      print("AP: ", ap)
  except:
    pass
  
  qs_res.append((q, duration, ap))

print(map40)



AP:  0.146
AP:  0.0
AP:  0.565
AP:  0.0
AP:  0.0
AP:  1.0
AP:  0.0
AP:  0.081
AP:  0.0
AP:  0.0
AP:  0.0
AP:  0.058
AP:  0.0
AP:  0.0
AP:  1.0
AP:  0.0
AP:  0.0
AP:  0.0
AP:  0.0
AP:  0.378
AP:  0.0
AP:  0.0
AP:  0.447
AP:  0.0
AP:  0.0
AP:  0.516
AP:  0.091
AP:  0.792
AP:  0.097
{'best marvel movie': [[0.022222222222222223, 0.044444444444444446], [0.125, 0.16666666666666666], 0.14583332604166702, 1.2330577373504639, 3], 'How do kids come to world?': [[], [], 0.0, 1.8834435939788818, 3], 'Information retrieval': [[0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2, 0.225, 0.25, 0.275, 0.3, 0.325, 0.35, 0.375, 0.4, 0.425, 0.45, 0.475, 0.5, 0.525, 0.55], [0.5, 0.3333333333333333, 0.42857142857142855, 0.5, 0.45454545454545453, 0.46153846153846156, 0.5, 0.5333333333333333, 0.5625, 0.5882352941176471, 0.6111111111111112, 0.631578947368421, 0.65, 0.6666666666666666, 0.6521739130434783, 0.6666666666666666, 0.6296296296296297, 0.6, 0.6129032258064516, 0.6060606060606061, 0.6176470588235294, 0.62

In [45]:
arr = list(map40.items())
length = len(list(map40.items()))
print("lenght: ", length)

def avg_map40():
  sum_a = 0
  sum_d = 0
  for k,v in arr:
    sum_a += v[2]
    sum_d += v[3]
  print("sum: ", sum_a)
  avg = sum_a / length
  avg_duration = sum_d / length
  return avg, avg_duration

print("AVG MAP40: " , avg_map40())

lenght:  29
sum:  5.17263584794386
AVG MAP40:  (0.17836675337737448, 0.7315384272871346)


In [33]:
print(qs_res[0])

('best marvel movie', 2.5176920890808105, 0.142)


In [22]:
import pandas as pd

new = pd.DataFrame.from_dict(map40)
new.to_csv("summary_search_bm25_top10_(1,0.5,0.5):_text:0.5_title:0.5_.csv")
new

,How do kids come to world?,Information retrieval,LinkedIn,How to make coffee?,Ritalin,How to make wine at home?,Most expensive city in the world,India,how to make money fast?,Netflix,...,Marijuana,How to make hummus,Winter,Rick and Morty,Natural Language processing,World Cup 2022,Dolly the sheep,What is the best place to live in?,Elon musk,How do you breed flowers?
0,[0.02],"[0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, ...","[0.06666666666666667, 0.13333333333333333, 0.2...",[0.021739130434782608],"[0.02631578947368421, 0.05263157894736842, 0.0...","[0.02, 0.04]",[0.021739130434782608],[],"[0.030303030303030304, 0.06060606060606061, 0....","[0.029411764705882353, 0.058823529411764705, 0...",...,"[0.02127659574468085, 0.0425531914893617, 0.06...","[0.022727272727272728, 0.045454545454545456, 0...","[0.02127659574468085, 0.0425531914893617, 0.06...","[0.034482758620689655, 0.06896551724137931, 0....","[0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, ...","[0.037037037037037035, 0.07407407407407407, 0....","[0.024390243902439025, 0.04878048780487805, 0....",[],"[0.043478260869565216, 0.08695652173913043, 0....",[0.020833333333333332]
1,[0.25],"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",[0.2],"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.875, 0.888888...","[0.3333333333333333, 0.25]",[1.0],[],"[1.0, 1.0, 0.75, 0.8]","[1.0, 1.0, 1.0, 0.8, 0.8333333333333334]",...,"[0.3333333333333333, 0.2857142857142857, 0.375]","[1.0, 1.0, 0.75, 0.8, 0.8333333333333334, 0.85...","[0.5, 0.6666666666666666, 0.75, 0.5]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8]","[1.0, 1.0, 1.0, 0.8, 0.5555555555555556, 0.6]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.88888888...",[],"[0.5, 0.6666666666666666, 0.75, 0.571428571428...",[0.1]
2,0.25,1.0,1.0,0.2,0.970486,0.291667,1.0,0.0,0.8875,0.926667,...,0.331349,0.873413,0.604167,1.0,0.975,0.825926,0.986111,0.0,0.60873,0.1
3,9.322633,0.986183,0.077322,1.506623,0.070289,3.54677,6.342253,0.847641,2.311913,0.117402,...,0.097133,1.394204,0.585588,0.205292,1.713948,4.300181,0.173613,5.478425,0.082921,0.31162
4,3,2,1,2,1,3,3,1,3,1,...,1,2,1,2,3,3,2,3,2,2


In [8]:
# import matplotlib.pyplot as plt

# queries = map(lambda x:)
# df = pd.DataFrame([[5, 2, 0], [3, 3, 2], [0, 1, 11]], classes, classes)
# new.plot(kind = 'scatter', x = 'Duration', y = 'Calories')

In [53]:
import requests

rs = requests.post('http://34.134.211.3:8080/get_pagerank', json=[123546,34568,85476])
print(rs.json())
rs2 = requests.post('http://34.134.211.3:8080/get_pageview', json=[123546,34568,85476])
print(rs2.json())

[0.7456973292487823, 14.897957134981224, 1.6932517377039191]
[530, 14284, 1878]


In [41]:
print(qs_res[13])

('World cup', 4.237330675125122, 1.0)


In [196]:
suma = 0.0
for q, duration, ap in qs_res:
  print(type(ap))
  if ap == None:
    continue
  suma = suma + ap

print(suma/len(qs_res))
print(qs_res)

<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'NoneType'>
<class 'float'>
<class 'float'>
<class 'float'>
0.4485666666666665
[('best marvel movie', 30.273027658462524, 0.12), ('How do kids come to world?', 4.396238327026367, 0.194), ('Information retrieval', 1.0254058837890625, 0.923), ('LinkedIn', 0.09195780754089355, 0.839), ('How to make coffee?', 1.5571134090423584, 0.417), ('Ritalin', 0.08665728569030762, 0.757), ('How to make wine at home?', 3.640566110610962, 0.28), ('Most expensive city in the world', 6.6025166511535645, 0.094), ('India', 0.8628292083740234, 0.07), ('how to make money fast?', 2.3640599250793

In [ ]:
!pip install PyDictionary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached PyDictionary-2.0.1-py3-none-any.whl (6.1 kB)
  Using cached goslate-1.5.4.tar.gz (14 kB)
  Preparing metadata (setup.py) ... done
  Using cached futures-3.0.5.tar.gz (25 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import PyDictionary #this is for our addition for the new method. dictionary that will help us in query expansion.
dictionary = PyDictionary()

query = "home is big and the apple is red"
tokens = tokenize(query.lower())
query_after_expansion = []

for token in tokens:
    try:
        synonym = dictionary.synonym(token)
        query_after_expansion.append(synonym[:3])
    except:
        pass

print("tokens: ", tokens)
print("query_after_expansion: ", query_after_expansion)
tokens += query_after_expansion
tokens

ModuleNotFoundError: ignored

In [ ]:
import requests
from time import time
url = 'http://34.134.211.3:8080'
# place the domain you got from ngrok or GCP IP below. 
# url = 'http://XXXX-XX-XX-XX-XX.ngrok.io'

qs_res = []
arr = list(queries.items())
for i in range(len(arr)-15):
  q, true_wids = arr[i][0],arr[i][1]
# for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search_body', {'query': q}, timeout=35)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
      #our method map 40
      map40_calculate(q,true_wids, pred_wids)
      #################
      print("AP: ", ap)
  except:
    pass
  
  qs_res.append((q, duration, ap))

print(map40)

AP:  0.333
AP:  0.654
AP:  1.0
AP:  0.75
AP:  0.962
AP:  0.5
AP:  0.375
AP:  0.2
AP:  0.869
AP:  0.421
AP:  0.25
AP:  0.333
{'best marvel movie': [[0.022222222222222223], [0.3333333333333333], 0.3333333333333333], 'How do kids come to world?': [[], [], 0], 'Information retrieval': [[0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175], [0.5, 0.6666666666666666, 0.75, 0.6666666666666666, 0.625, 0.6666666666666666, 0.7], 0.6535714285714285], 'LinkedIn': [[0.06666666666666667, 0.13333333333333333, 0.2, 0.26666666666666666, 0.3333333333333333, 0.4, 0.4666666666666667], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 1.0], 'How to make coffee?': [[0.021739130434782608, 0.043478260869565216, 0.06521739130434782, 0.08695652173913043], [1.0, 1.0, 0.5, 0.5], 0.75], 'Ritalin': [[0.02631578947368421, 0.05263157894736842, 0.07894736842105263, 0.10526315789473684, 0.13157894736842105, 0.15789473684210525, 0.18421052631578946], [1.0, 1.0, 1.0, 1.0, 1.0, 0.8571428571428571, 0.875], 0.961734693877551], 'How to make wine

In [ ]:
arr = list(map40.items())
length = len(list(map40.items()))
print("lenght: ", length)

def avg_map40():
  sum_a = 0
  for k,v in arr:
    sum_a += v[2]
  print("sum: ", sum_a)
  avg = sum_a / length
  return avg

print("AVG MAP40: " , avg_map40())

lenght:  15
sum:  6.64705215419501
AVG MAP40:  0.44313681027966734
